## Background

In order to manage all useful information and data in these recordings more effectively, I created a class structure called `cage_data`. Each `.pkl` file in the folder `continuous` contains one `cage_data` object, corresponding to one of the 15-minute long recording files. For some practical reasons, we have a 2-second long pause between every two adjacent 15-minute long recording files.

To read those `.pkl` files, first you need to have the codes defining the `cage_data` object in your path. You can find those codes in this [repo](https://github.com/limblab/cage_data). Clone it to your local machine, and simply add it to your path like this:

In [1]:
import fnmatch, os, sys
sys.path.append('path to the cloned repo/cage_data')

In [2]:
import cage_data
import numpy as np

## Load one file as an example

Data are stored in the variable `my_cage_data`, an instance of the `cage_data` class.

In [3]:
import pickle

data_path = 'E:/Box Sync/Pop_20201020/cage/50ms/continuous/'
file_name = '20201020_Pop_Cage_003.pkl'
print('The file %s is going to be loaded'%(data_path + file_name))
with open ( data_path + file_name, 'rb' ) as fp:
    my_cage_data = pickle.load(fp)
my_cage_data.pre_processing_summary()

The file E:/Box Sync/Pop_20201020/cage/50ms/continuous/20201020_Pop_Cage_003.pkl is going to be loaded
This is a non-sorted file
EMG filtered? -- True
EMG filtered? -- True
Cortical data cleaned? -- True
Data binned? -- True
Spikes smoothed? -- True


## Some basic information about the file just loaded

In [4]:
print('There are %d cortical channels'%(len(my_cage_data.spikes)))
print('There are %d EMG channels'%(len(my_cage_data.EMG_diff)))
print('The raw EMG signals are sampled at %.3f Hz'%(my_cage_data.EMG_fs))
print('There are %d behavior segments in this file'%(len(my_cage_data.behave_tags['tag'])))
print('The length of this file is %.3f seconds'%(my_cage_data.EMG_timeframe[-1]))
print('Spikes and EMGs are binned or downsampled with %.2f seconds time bins'%(my_cage_data.binned['timeframe'][1]-my_cage_data.binned['timeframe'][0]))

There are 72 cortical channels
There are 15 EMG channels
The raw EMG signals are sampled at 2011.061 Hz
There are 124 behavior segments in this file
The length of this file is 900.108 seconds
Spikes and EMGs are binned or downsampled with 0.05 seconds time bins


## How to get raw EMGs?

The raw EMGs are acquired by DSPW wireless system with an Intan RHD2132 frontend. Since the channels on RHD2132 are all single-ended, we do software differential after getting the signals. Therefore, here the raw EMGs are stored in a field called `EMG_diff`.

More specifically, `EMG_diff` is an attribute of the `cage_data` class, and could be accessed by calling:
* raw_EMGs = my_cage_data.EMG_diff

And the sampling frequency of the raw EMGs could be obtained by calling:
* fs_raw_EMG = my_cage_data.EMG_fs

The time frame of the raw EMGs could be obtained by calling:
* raw_EMG_timeframe = my_cage_data.EMG_timeframe

The names of each EMG channel could be got by:
* EMG_names = my_cage_data.EMG_names

In [5]:
raw_EMGs = my_cage_data.EMG_diff
print('There are %d channels, and each channel has %d time samples'%(len(raw_EMGs), len(raw_EMGs[0])))

raw_EMG_timeframe = my_cage_data.EMG_timeframe

fs_raw_EMG = my_cage_data.EMG_fs
print('The raw EMG signals are sampled %.3f Hz'%(fs_raw_EMG))

EMG_names = my_cage_data.EMG_names
print(EMG_names)

There are 15 channels, and each channel has 1810172 time samples
The raw EMG signals are sampled 2011.061 Hz
['APB', 'Lum', 'PT', 'FDP2', 'FCR1', 'FCU1', 'FCUR', 'FPB', '3DI', 'SUP', 'ECU', 'ECR', 'EDC1', 'BI', 'TRI']


Here `raw_EMGs` is a list

## How to get raw spike timings?

Raw spike timings are stored with the attribute `spikes`, and could be got like this:
* spike_timing = my_cage_data.spikes

In [6]:
spike_timing = my_cage_data.spikes

Here `spike_timing` is a list

## How to get binned / downsampled spike counts and filtered EMG envelops? 

The binned data is stored with another attribute named `binned`. It is a dictionary, and could be accessed like this:
* binned = my_cage_data.binned

There are 4 fields in this dictionary:
* spikes: binned spike counts
* filtered_EMG: EMG envelops been rectified, filtered and downsampled
* FSR_data: the data from force sensitive resistors inside the plastic cage, only meaningful for power grasping
* timeframe: the common time frame for the 3 types of data above

Data in the files we are using now are binned with 50 ms time bins

In [7]:
print(my_cage_data.binned.keys())

# To get the binned spike counts
binned_spike_counts = my_cage_data.binned['spikes']

# To get the rectified, filtered and downsampled EMGs
filtered_EMG = my_cage_data.binned['filtered_EMG']

# To get the time frame of the binned data
timeframe = my_cage_data.binned['timeframe']

dict_keys(['timeframe', 'spikes', 'filtered_EMG', 'FSR_data'])


## How to get the information about the monkey's behavior?
The start time and end time and the type of the behaivor of each behavior segment are stored with the attribute `behave_tags`, which is also a dictionary. It has 3 fields, and the names of them are self-explanatory:

* start_time
* end_time
* tag

For example, check the 74th behavior segment in this file:

In [8]:
N = 74
print('During the %dth behavior segment, the monkey was doing \'%s\''%(N, my_cage_data.behave_tags['tag'][N]))
print('The start time of the %dth behavior segment is at the %.3f second'%(N, my_cage_data.behave_tags['start_time'][N]))
print('The end time of the %dth behavior segment is at the %.3f second'%(N, my_cage_data.behave_tags['end_time'][N]))

During the 74th behavior segment, the monkey was doing 'pg'
The start time of the 74th behavior segment is at the 468.710 second
The end time of the 74th behavior segment is at the 470.276 second


Note the index of the first behavior segment is "_0_"

In fact, the behavior segments stored in the files I shared last time are generated by the function below. But since we already have them, further introduction is not needed here

In [9]:
behavior_segs = my_cage_data.get_all_data_segment(requires_raw_EMG = True, requires_spike_timing = True)
# You can save behavior_segs to the disk if you would like

## A function to filter the raw EMGs
A number of useful functions are designed as the methods of the `cage_data` behavior. This one is used to filter the raw EMGs and then to get the EMG envelops
* EMG_filtering(self, f_Hz), `f_Hz` is the corner frequency of the low pass filter to get the envelops

There is no return from this function. This function will update (or create if not existing) the attribute `filtered_EMG`.

Note that here `filtered_EMG` is an attribute of the `cage_data` class, which is different from the field `filtered_EMG` under the attribute `binned`


In [13]:
# Filtering the raw EMGs with a 15 Hz low-pass filter and to get the rectified envelops, all done with this function
my_cage_data.EMG_filtering(15)
EMG_envelops = my_cage_data.filtered_EMG

All EMG channels have been filtered.


As no downsampling or binning is performed here, the obtained EMG envelops are still have the same sampling frequency as the raw EMGs, which is `my_cage_data.EMG_fs` as mentioned above.

## A function to bin the spikes and downsample the EMGs
Although the files shared in Box are all binned with 50 msec bins, you can still use this function to re-bin the data. After doing this, the items in the attribute `binned` will be updated.
* bin_data(self, bin_size, mode = 'center')

`bin_size` is the size of the bin in seconds. `mode` is the way to align the time bins, `center` is OK, and another option is `left`

This function will not return a value either. It only update the attribute `binned`

In [14]:
new_bin_size = 0.02 # 20 msec
my_cage_data.bin_data(new_bin_size)

Binning spikes with 0.0200 s
Filtered EMGs have been downsampled
FSR data have been downsampled
Data have been binned.


## More...

By calling the function `save_to_pickle` like this:
* my_cage_data.save_to_pickle(save_path, save_file_name)

You could save the instance you updated.

If the `my_cage_data` instance is occupying too much memory, just delete it like this
* del(my_cage_data)

You could pull different types of data from it and save them individually before deleting it